In [1]:
import sys
from pathlib import Path
import os

# Get the current working directory (where notebook is executed from)
current_dir = Path.cwd()

# Start from current directory and search upward for project root
# Project root should contain pyproject.toml (not in src/)
project_root = current_dir
max_levels = 10  # Safety limit

for _ in range(max_levels):
    
    # Check if this directory contains pyproject.toml
    if (project_root / "pyproject.toml").exists():
        # Verify it's the actual project root (not a subdirectory)
        # Project root should have pyproject.toml and src/ directory
        if (project_root / "src").exists() and (project_root / "pyproject.toml").exists():
            break
    if project_root == project_root.parent:
        # Reached filesystem root
        break
    project_root = project_root.parent
else:
    # Fallback: go up 3 levels from current directory if we're in src/adapters/ai_chat/
    if "src" in str(current_dir) and "adapters" in str(current_dir):
        project_root = current_dir.parent.parent.parent

# Add project root to Python path (must be absolute path)
project_root = project_root.resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Change working directory to project root
os.chdir(project_root)

print(f"Current directory (before): {current_dir}")
print(f"Project root: {project_root}")
print(f"Working directory (after): {os.getcwd()}")
print(f"pyproject.toml exists: {(project_root / 'pyproject.toml').exists()}")
print(f"src/ exists: {(project_root / 'src').exists()}")


Current directory (before): f:\Dev\interview-service\interview-service\src\adapters\ai_chat
Project root: F:\Dev\interview-service\interview-service
Working directory (after): F:\Dev\interview-service\interview-service
pyproject.toml exists: True
src/ exists: True


In [2]:
from openai import OpenAI
from src.adapters.ai_chat.ai_chat import AIChat
from src.domain.vacancy.vacancy import VacancyInfo
from src.domain.message.message import Message, RoleEnum, TypeEnum
from src.domain.task.task import Task, TaskType
import asyncio
import dotenv   
import os
from config import MODEL_NAME, TOKEN_LIMIT

dotenv.load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
# Вариант с доменом без порта (HTTPS):
BASE_URL = "https://llm.t1v.scibox.tech/v1"
# Альтернатива с IP:порт
# BASE_URL = "http://45.145.191.148:4000/v1"

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)
print(client)

In [3]:
print(MODEL_NAME, TOKEN_LIMIT)

qwen3-32b-awq 25000


In [4]:
test_interview_plan = """INTERNAL INTERVIEW PLAN ONLY - DO NOT SHARE WITH CANDIDATES  

1. **Warm-up Question (5-7 min)**  
   - [theory] *Explain the primary use cases for Pandas DataFrames vs. NumPy ndarrays. When would you choose one over the other?*  

2. **Theoretical Questions (10-12 min)**  
   - [theory] *What is the purpose of SQLAlchemy's ORM layer? How does it simplify database interactions compared to raw SQL?*  
   - [theory] *Compare TensorFlow and PyTorch. In what scenarios is each framework typically preferred?*  

3. **Core Coding Tasks (30-35 min)**  
   - **Task 1** [coding] *Write Pandas code to load a CSV file, filter rows where column 'A' > 10, and calculate the mean of column 'B'.*  
   - **Task 2** [coding] *Create a NumPy array of shape (3,3) filled with random values. Compute eigenvalues and perform matrix inversion.*  
   - **Task 3** [coding] *Build a simple neural network (1 hidden layer) using PyTorch/TensorFlow to classify the Iris dataset (skeleton code provided). Compile and explain the model.*  

4. **Follow-up & Debugging (10-12 min)**  
   - [theory] *Explain how you would optimize the Pandas code for large datasets.*  
   - [coding] *Debug a provided SQLAlchemy ORM query that fails to join two tables correctly.*  

5. **Wrap-up (3-5 min)**  
   - [theory] *What are the key challenges when integrating NumPy/TensorFlow for GPU-accelerated computations?*  

---  
**Timing Notes**: Adjust based on candidate performance. Prioritize depth in core libraries (Pandas, NumPy) over framework specifics."""

ai_message_content_1 = """Здравствуйте! Добро пожаловать на техническое интервью.  
В ходе интервью вы будете решать задачи на Python (Pandas, Numpy, PyTorch и др.) в встроенной среде. Время ограничено, поэтому работайте аккуратно и оперативно.  
Вы можете задавать уточняющие вопросы по условию задач, но не ожидайте, что я напишу решение за вас — я помогу направить вас в правильное русло. Мы также обсудим ваш подход и код.  
**Важно:** не используйте внешние инструменты (LLM, поисковики), не копируйте код извне — всё должно быть введено вручную. Попытки обойти правила приведут к дезавалидации интервью.  
Чат будет проверен после завершения. Начнём с первой задачи — готовы?
"""

In [8]:
import asyncio
from src.domain.vacancy.vacancy import VacancyInfo
from src.domain.message.message import Message, RoleEnum, TypeEnum
from src.domain.task.task import Task, TaskType
from datetime import timedelta

# Create example vacancy info
vacancy_info = VacancyInfo(
    profession="Python разработчик / Data Scientist",
    position="Junior Python Developer",
    requirements="Pandas, Numpy, Tensorflow, PyTorch, SQLAlchemy",
    questions="",
    tasks=None,
    task_ides=None,
    interview_plan=test_interview_plan,  # Will be generated by create_chat
    duration=timedelta(minutes=30)
)

# Create example chat history
chat_history = [
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.RESPONSE,
        content=ai_message_content_1
    ),
    Message(
        role=RoleEnum.AI,
        type=TypeEnum.QUESTION,
        content="Поясните основные случаи использования Pandas DataFrames и NumPy ndarrays. В каких ситуациях вы выберете один инструмент вместо другого?"
    ),
    Message(
        role=RoleEnum.USER,
        type=TypeEnum.OTHER,
        content="А... можете пояснить вопрос? Напомните, что такое ndarray?"
    )
]


ai_chat = AIChat()



In [9]:
# Test create_response function
async def test_create_response():
    ai_chat = AIChat()
    
    # Example task for this interview turn
    task = Task(
        type=TaskType.THEORY,
        language="Python",
        description="Поясните основные случаи использования Pandas DataFrames и NumPy ndarrays. В каких ситуациях вы выберете один инструмент вместо другого?"
    )

    stream, user_msg, ai_msg = await ai_chat.create_response(
        vacancy_info=vacancy_info,
        chat_history=chat_history,
        task=task,
    )

    # Fill user message content from the last USER message in history
    last_user_msg = next((m for m in reversed(chat_history) if m.role == RoleEnum.USER), None)
    if last_user_msg:
        user_msg.content = last_user_msg.content

    print("=== Classification ===")
    print(f"User message type:      {user_msg.type}")
    print(f"Assistant message type: {ai_msg.type}")

    print("\n=== AI response (streaming) ===\n")
    chunks: list[str] = []
    async for chunk in stream:
        chunks.append(chunk)
        print(chunk, end="", flush=True)

    ai_msg.content = "".join(chunks)

    print("\n\n=== Final AI message ===")
    print(f"Role:    {ai_msg.role}")
    print(f"Type:    {ai_msg.type}")
    print(f"Content: {ai_msg.content}")


print("Testing create_response...")
await test_create_response()


Testing create_response...
=== Classification ===
User message type:      question
Assistant message type: hint

=== AI response (streaming) ===

  
NumPy ndarray (N-мерный массив) — это основной тип данных в NumPy для работы с числовыми данными, оптимизированный для высокой производительности. Он представляет собой однородную структуру (все элементы одного типа) и поддерживает векторизованные операции. Pandas DataFrame — это структура, похожая на таблицу в Excel, с поддержкой разнотиповых колонок, индексации и операций, характерных для работы с данными (фильтрация, группировка и т.д.).  

Попробуйте сформулировать, в каких сценариях использование одного из этих инструментов предпочтительнее. Например: когда нужно выполнять математические вычисления над массивами данных — NumPy, а когда работать с табличными данными, содержащими разнородные типы — Pandas.

=== Final AI message ===
Role:    ai
Type:    hint
Content:   
NumPy ndarray (N-мерный массив) — это основной тип данных в NumPy дл

In [11]:
stream

NameError: name 'stream' is not defined

In [8]:

# Test create_chat function
async def test_create_chat():
    ai_chat = AIChat()
    
    # create_chat returns updated VacancyInfo with interview_plan
    updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    
    print("=== Updated Vacancy Info ===")
    print(f"Profession: {updated_vacancy.profession}")
    print(f"Position: {updated_vacancy.position}")
    print(f"\nInterview Plan (first 300 chars):")
    print(updated_vacancy.interview_plan + "..." if len(updated_vacancy.interview_plan) > 300 else updated_vacancy.interview_plan)
    print(f"\nFull Interview Plan Length: {len(updated_vacancy.interview_plan)} characters")

# Test generate_welcome_message function
async def test_generate_welcome_message():
    ai_chat = AIChat()
    
    # First create chat to get updated vacancy with interview_plan
    updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    
    print("\n=== Welcome Message (streaming) ===\n")
    welcome_chunks = []
    
    # ❌ before:
    # async for chunk in ai_chat.generate_welcome_message(updated_vacancy, chat_history):

    # ✅ after:
    stream = await ai_chat.generate_welcome_message(updated_vacancy, chat_history)
    async for chunk in stream:
        welcome_chunks.append(chunk)
        print(chunk, end="", flush=True)
    
    print("\n\n=== Full Welcome Message ===")
    full_welcome = "".join(welcome_chunks)
    print(full_welcome)

# Run the async functions
print("Testing create_chat...")
await test_create_chat()

print("\n" + "="*50 + "\n")

print("Testing generate_welcome_message...")
await test_generate_welcome_message()


Testing create_chat...
=== Updated Vacancy Info ===
Profession: Python разработчик / Data Scientist
Position: Junior Python Developer

Interview Plan (first 300 chars):
INTERNAL INTERVIEW PLAN ONLY (DO NOT SHARE WITH CANDIDATE)

1. **Warm-up (5-10 min)**  
   - [theory] Explain how to handle missing data in Pandas DataFrames. Demonstrate .dropna() vs .fillna() with example scenarios.  

2. **Core Libraries Assessment (30-40 min)**  
   - [coding] Numpy task: Write a function to normalize a 2D array (row-wise) and handle division by zero.  
   - [theory] Compare Tensorflow and PyTorch: When would you choose one over the other? Discuss dynamic vs static computation graphs.  

3. **ML Frameworks Practical (20-30 min)**  
   - [coding] PyTorch task: Implement a single-layer neural network for binary classification (define model, loss function, and optimizer).  

4. **Database Integration (15-20 min)**  
   - [theory] Explain SQLAlchemy's ORM approach. How does it differ from raw SQL? Provi

In [ ]:
# Test create_response function (for reference)
async def test_create_response():
    ai_chat = AIChat()
    
    # First create chat to get updated vacancy
    updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    
    # Then create response with task
    print("\n=== Response to Task (streaming) ===\n")
    response_chunks = []
    stream = await ai_chat.create_response(updated_vacancy, chat_history, task)
    async for chunk in stream:
        response_chunks.append(chunk)
        print(chunk, end="", flush=True)
    
    print("\n\n=== Full Response ===")
    full_response = "".join(response_chunks)
    print(full_response)

# Run the async function
await test_create_response()


In [6]:
# Test stream_task function
async def test_stream_task():
    ai_chat = AIChat()
    
    # First create chat to get updated vacancy with interview_plan
    # updated_vacancy = await ai_chat.create_chat(vacancy_info, chat_history)
    vacancy_info.interview_plan = test_interview_plan
    updated_vacancy = vacancy_info
    print(updated_vacancy.interview_plan)
    # Then stream the task
    print("\n=== Streaming Task Description ===\n")
    task_chunks = []
    
    stream = await ai_chat.stream_task(updated_vacancy, chat_history)
    async for chunk in stream:
        task_chunks.append(chunk)
        print(chunk, end="", flush=True)
    
    print("\n\n=== Full Task Description ===")
    full_task = "".join(task_chunks)
    print(full_task)

# Run the async function
await test_stream_task()


INTERNAL INTERVIEW PLAN ONLY - DO NOT SHARE WITH CANDIDATES  

1. **Warm-up Question (5-7 min)**  
   - [theory] *Explain the primary use cases for Pandas DataFrames vs. NumPy ndarrays. When would you choose one over the other?*  

2. **Theoretical Questions (10-12 min)**  
   - [theory] *What is the purpose of SQLAlchemy's ORM layer? How does it simplify database interactions compared to raw SQL?*  
   - [theory] *Compare TensorFlow and PyTorch. In what scenarios is each framework typically preferred?*  

3. **Core Coding Tasks (30-35 min)**  
   - **Task 1** [coding] *Write Pandas code to load a CSV file, filter rows where column 'A' > 10, and calculate the mean of column 'B'.*  
   - **Task 2** [coding] *Create a NumPy array of shape (3,3) filled with random values. Compute eigenvalues and perform matrix inversion.*  
   - **Task 3** [coding] *Build a simple neural network (1 hidden layer) using PyTorch/TensorFlow to classify the Iris dataset (skeleton code provided). Compile and ex